In [12]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from utils.embedding_utils import *
from utils.classifier_utils import *
from classifiers.CustomClassifiers import AggregateClassifier,AggregateClassifierGrid
from itertools import combinations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
embedding_dir=os.path.join(os.getcwd(),'outputs',"stats","main_54")
full_stat_df=pd.read_csv(os.path.join(embedding_dir,"embedding_stats_avg5.csv"))
output_dir=os.path.join(embedding_dir,"stat_analysis")

In [14]:
def run_embedding_classiciation(stat_df,X_weight,use_prob=True,min_combos=3,
                                max_combos=4,skip_w=True,pred_col='diagnosis',conditionals=[]):
    graph_labels = stat_df[pred_col].values
    data_names=['W','rad','btw','coh']
#     data_names=['rad','btw','coh']
    features=[]
    for n in data_names:
        if n in ['rad','btw','coh']:
            cols_to_use=[c for c in stat_df.columns if n in c.lower()]
            if use_prob:
                cols_to_use=[c for c in cols_to_use if 'prob' in c.lower()]
            else:
                cols_to_use=[c for c in cols_to_use if 'prob' not in c.lower()]
            n_df=np.array(stat_df[cols_to_use])
            data,y_comb,pat_to_indx=combine_patient_feats(n_df,graph_labels,stat_df,conditional=conditionals)
            print(n_df,'NF')
        else:
            n_df=X_weight
            data,y_comb,pat_to_indx=combine_patient_feats(n_df,graph_labels,stat_df,conditional=conditionals)
            Wa=data
            
            
#         data,y_comb,pat_to_indx=combine_patient_feats(n_df,graph_labels,stat_df,conditional=conditionals)
        features.append(data)
        
    
        

#     Wa=X_weight_comb
#     names=['W','X_rc','X_bc','X_wc']
    indices=np.array([i for i in range(len(data_names))])
    feature_combinations=[]
    name_combinations=[]
    list_combinations = list()

    for n in range(min_combos,max_combos+1):
        for combo in combinations(indices, n):
            print(combo)
            feat_list=[features[i] for i in combo]
            name_list=[data_names[i] for i in combo]
            print(name_list)
            feat_combines=np.hstack(feat_list)
            print(feat_combines.shape)
            feature_combinations.append(feat_combines)
            name_combinations.append(name_list)

    if not skip_w:
        feature_combinations=[Wa]+feature_combinations
        name_combinations=[['W']]+name_combinations
        
    return feature_combinations,name_combinations,y_comb
    

In [15]:
conditionals = ((full_stat_df['CogTr']==1))
fit_method='concatenate'
predict_method='same'
# run_combos_list(model_type=LogisticRegression,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb)  


In [16]:
import argparse
parser = argparse.ArgumentParser(description='RiemannianGNN')
args, _ = parser.parse_known_args()
MEGLpParams.add_params(parser)
args = parser.parse_args(args=[])

In [17]:
BAND_TO_INDEX = {'theta':0,'alpha':1,'beta1':2,'beta2':3,'beta':4,'gamma':5}
METRIC_TO_INDEX = {'plv':0,'ciplv':1}


band_adj = BAND_TO_INDEX['alpha']
metric_adj = METRIC_TO_INDEX[args.metric]
raw_scans = np.load(args.raw_scan_file)
scan_data = raw_scans[:,band_adj,metric_adj] ### this is not changed by any hyperparamters

graph_ids = full_stat_df['Scan Index'].values
scan_data = raw_scans[:,band_adj,metric_adj] ### this is not changed by any hyperparamters
train_scans = scan_data[graph_ids.astype(int) ]
X_weight = np.array([a[np.triu_indices(a.shape[0])] for a in train_scans])

In [25]:

feature_combinations,name_combinations,y_comb=run_embedding_classiciation(full_stat_df,X_weight,skip_w=True,
                                                                   use_prob=False,min_combos=1,max_combos=4,conditionals=conditionals)

conditational
0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    False
179    False
Name: CogTr, Length: 180, dtype: bool
NO SHUFFLE
conditational
0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    False
179    False
Name: CogTr, Length: 180, dtype: bool
NO SHUFFLE
[[1.14856664 2.0953061  2.18031072 ... 2.05160935 1.42537626 2.26457091]
 [1.12309426 2.06000353 2.21085388 ... 2.09441905 1.45372466 2.33475558]
 [1.21732124 1.98617806 2.04157465 ... 1.89624364 1.33290461 1.99036129]
 ...
 [1.19672428 2.10935528 2.21695379 ... 2.04229307 1.52801455 2.00044454]
 [1.30283475 1.89239307 1.91787272 ... 1.98593059 1.42585129 1.99628657]
 [1.29968796 1.78858962 1.84196672 ... 1.9000061  1.39210995 1.90546139]] NF
conditational
0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    Fal

In [8]:
# feature_combinations,name_combinations,y_comb=run_embedding_classiciation(all_stat_dicts[0],X_weight,
#                                                                    use_prob=True,min_combos=3,max_combos=4)

In [19]:
for f in feature_combinations:
    print(f.shape)

(46, 4095, 2)
(46, 4103, 2)
(46, 4123, 2)
(46, 4103, 2)
(46, 36, 2)
(46, 16, 2)
(46, 36, 2)
(46, 4131, 2)
(46, 4111, 2)
(46, 4131, 2)
(46, 44, 2)
(46, 4139, 2)


In [30]:
fit_method='split_sample'
predict_method='prediction_mean'
run_combos_list(model_type=LinearRegression,names=name_combinations,
                feature_list=feature_combinations,times=10,
                use_agg=True,agg_fit=fit_method,
                agg_pred=predict_method,y_comb=y_comb,use_nested=False)  
# run_combos_list(model_type=SVC,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,
#                 agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

0.684014985014985 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4095, 2) new
(46,) clone
(46, 4095, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.74      0.64      0.68        22
           2       0.70      0.79      0.75        24

    accuracy                           0.72        46
   macro avg       0.72      0.71      0.71        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.7348484848484849 roc
(0.7140124342419896, 0.7348484848484849) res
0.678947607947608 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1

0.6676466866466866 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 36, 2) new
(46,) clone
(46, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.71      0.68      0.70        22
           2       0.72      0.75      0.73        24

    accuracy                           0.72        46
   macro avg       0.72      0.72      0.72        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.7196969696969696 roc
(0.7161841480778358, 0.7196969696969696) res
0.715620657120657 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2

0.6587365412365414 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 8, 2) new
(46,) clone
(46, 8, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.73      0.50      0.59        22
           2       0.65      0.83      0.73        24

    accuracy                           0.67        46
   macro avg       0.69      0.67      0.66        46
weighted avg       0.69      0.67      0.66        46

0.6739130434782609 accuracy
0.7348484848484848 roc
(0.6609336609336609, 0.7348484848484848) res
0.6932761682761681 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 

0.6708387168387168 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 44, 2) new
(46,) clone
(46, 44, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.72      0.59      0.65        22
           2       0.68      0.79      0.73        24

    accuracy                           0.70        46
   macro avg       0.70      0.69      0.69        46
weighted avg       0.70      0.70      0.69        46

0.6956521739130435 accuracy
0.7178030303030303 roc
(0.6903846153846154, 0.7178030303030303) res
0.7129580419580419 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2

0.6817700077700077 50 num_split?
{'C': 200, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 36, 2) new
(46,) clone
(46, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.77      0.77      0.77        22
           2       0.79      0.79      0.79        24

    accuracy                           0.78        46
   macro avg       0.78      0.78      0.78        46
weighted avg       0.78      0.78      0.78        46

0.782608695652174 accuracy
0.821969696969697 roc
(0.7821969696969697, 0.821969696969697) res
0.6742695082695083 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 

...

0.6551767121767123 50 num_split?
{'C': 0.3, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 44, 2) new
(46,) clone
(46, 44, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.67      0.64      0.65        22
           2       0.68      0.71      0.69        24

    accuracy                           0.67        46
   macro avg       0.67      0.67      0.67        46
weighted avg       0.67      0.67      0.67        46

0.6739130434782609 accuracy
0.6761363636363636 roc
(0.6725201708590414, 0.6761363636363636) res
0.7181025086025086 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 

0.6859574869574868 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 36, 2) new
(46,) clone
(46, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.71      0.68      0.70        22
           2       0.72      0.75      0.73        24

    accuracy                           0.72        46
   macro avg       0.72      0.72      0.72        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.7121212121212122 roc
(0.7161841480778358, 0.7121212121212122) res
0.6571717171717171 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 

0.6194508824508824 50 num_split?
{'C': 0.3, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 8, 2) new
(46,) clone
(46, 8, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.69      0.50      0.58        22
           2       0.63      0.79      0.70        24

    accuracy                           0.65        46
   macro avg       0.66      0.65      0.64        46
weighted avg       0.66      0.65      0.64        46

0.6521739130434783 accuracy
0.706439393939394 roc
(0.6413255360623782, 0.706439393939394) res
0.682336052836053 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1

0.7122740037740036 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4111, 2) new
(46,) clone
(46, 4111, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.73      0.73      0.73        22
           2       0.75      0.75      0.75        24

    accuracy                           0.74        46
   macro avg       0.74      0.74      0.74        46
weighted avg       0.74      0.74      0.74        46

0.7391304347826086 accuracy
0.7234848484848484 roc
(0.7386363636363636, 0.7234848484848484) res
0.7016035631035631 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1

0.7117506382506381 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'degree': 3, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4123, 2) new
(46,) clone
(46, 4123, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.71      0.68      0.70        22
           2       0.72      0.75      0.73        24

    accuracy                           0.72        46
   macro avg       0.72      0.72      0.72        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.7518939393939393 roc
(0.7161841480778358, 0.7518939393939393) res
0.725697691197691 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1

In [29]:
fit_method='concatenate'
predict_method='same'
run_combos_list(model_type=SVC,names=name_combinations,
                feature_list=feature_combinations,times=10,
                use_agg=True,agg_fit=fit_method,
                agg_pred=predict_method,y_comb=y_comb,use_nested=False)  
# run_combos_list(model_type=SVC,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,
#                 agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

0.6335339660339661 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4095, 2) new
(46,) clone
(46, 4095, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.68      0.68      0.68        22
           2       0.71      0.71      0.71        24

    accuracy                           0.70        46
   macro avg       0.70      0.70      0.70        46
weighted avg       0.70      0.70      0.70        46

0.6956521739130435 accuracy
0.7140151515151515 roc
(0.6950757575757576, 0.7140151515151515) res
0.6427227772227773 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1

0.6804316239316239 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'degree': 3, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 36, 2) new
(46,) clone
(46, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.76      0.73      0.74        22
           2       0.76      0.79      0.78        24

    accuracy                           0.76        46
   macro avg       0.76      0.76      0.76        46
weighted avg       0.76      0.76      0.76        46

0.7608695652173914 accuracy
0.7803030303030303 roc
(0.7598481252966303, 0.7803030303030303) res
0.7122635142635143 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 

0.6597057387057387 50 num_split?
{'C': 0.3, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 8, 2) new
(46,) clone
(46, 8, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.75      0.55      0.63        22
           2       0.67      0.83      0.74        24

    accuracy                           0.70        46
   macro avg       0.71      0.69      0.69        46
weighted avg       0.71      0.70      0.69        46

0.6956521739130435 accuracy
0.7613636363636365 roc
(0.6861598440545809, 0.7613636363636365) res
0.696617993117993 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 

0.6633911088911089 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 44, 2) new
(46,) clone
(46, 44, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.76      0.59      0.67        22
           2       0.69      0.83      0.75        24

    accuracy                           0.72        46
   macro avg       0.73      0.71      0.71        46
weighted avg       0.73      0.72      0.71        46

0.717391304347826 accuracy
0.696969696969697 roc
(0.7106918238993711, 0.696969696969697) res
0.7018427128427127 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2

0.6729179154179153 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 36, 2) new
(46,) clone
(46, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.65      0.68      0.67        22
           2       0.70      0.67      0.68        24

    accuracy                           0.67        46
   macro avg       0.67      0.67      0.67        46
weighted avg       0.67      0.67      0.67        46

0.6739130434782609 accuracy
0.7405303030303031 roc
(0.6737588652482269, 0.7405303030303031) res
0.7117359862359862 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 

...

0.6657347652347652 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 44, 2) new
(46,) clone
(46, 44, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.76      0.59      0.67        22
           2       0.69      0.83      0.75        24

    accuracy                           0.72        46
   macro avg       0.73      0.71      0.71        46
weighted avg       0.73      0.72      0.71        46

0.717391304347826 accuracy
0.696969696969697 roc
(0.7106918238993711, 0.696969696969697) res
0.7195574425574427 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2

0.6750294705294705 50 num_split?
{'C': 0.3, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 36, 2) new
(46,) clone
(46, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.79      0.68      0.73        22
           2       0.74      0.83      0.78        24

    accuracy                           0.76        46
   macro avg       0.77      0.76      0.76        46
weighted avg       0.76      0.76      0.76        46

0.7608695652173914 accuracy
0.7462121212121212 roc
(0.7580105212816834, 0.7462121212121212) res
0.7044118104118104 50 num_split?
{'C': 200, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1

0.7065440115440116 50 num_split?
{'C': 200, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 8, 2) new
(46,) clone
(46, 8, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.64      0.73      0.68        22
           2       0.71      0.62      0.67        24

    accuracy                           0.67        46
   macro avg       0.68      0.68      0.67        46
weighted avg       0.68      0.67      0.67        46

0.6739130434782609 accuracy
0.8409090909090909 roc
(0.6737588652482269, 0.8409090909090909) res
0.6469765789765789 50 num_split?
{'C': 0.3, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1

0.7251421911421911 50 num_split?
{'C': 0.3, 'class_weight': 'balanced', 'degree': 3, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4111, 2) new
(46,) clone
(46, 4111, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.79      0.68      0.73        22
           2       0.74      0.83      0.78        24

    accuracy                           0.76        46
   macro avg       0.77      0.76      0.76        46
weighted avg       0.76      0.76      0.76        46

0.7608695652173914 accuracy
0.7215909090909092 roc
(0.7580105212816834, 0.7215909090909092) res
0.6929307914307914 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 

0.7185494505494506 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4123, 2) new
(46,) clone
(46, 4123, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.80      0.73      0.76        22
           2       0.77      0.83      0.80        24

    accuracy                           0.78        46
   macro avg       0.78      0.78      0.78        46
weighted avg       0.78      0.78      0.78        46

0.782608695652174 accuracy
0.7481060606060607 roc
(0.7809523809523811, 0.7481060606060607) res
0.6996332001332 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 

In [23]:
fit_method='split_sample'
predict_method='same'
run_combos_list(model_type=SVC,names=name_combinations,
                feature_list=feature_combinations,times=1,
                use_agg=True,agg_fit=fit_method,
                agg_pred=predict_method,y_comb=y_comb,use_nested=False)  
# run_combos_list(model_type=SVC,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,
#                 agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

0.795052872909684 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1 1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2
 1 1 2 2 2 2 2 2 1 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(92,) EXAMPLE Y
(92, 4095) new
(92,) clone
(92, 4095)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1 1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2
 1 1 2 2 2 2 2 2 1 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.82      0.75      0.79        44
           2       0.79      0.85      0.82        48

    accuracy                           0.80        92
   macro avg       0.81      0.80      0.80        92
weighted avg       0.81      0.80      0.80        92

0.8043478260869565 accuracy
0.8726325757575757 roc
(0.8028571428571428, 0.87263

              precision    recall  f1-score   support

           1       0.87      0.77      0.82        44
           2       0.81      0.90      0.85        48

    accuracy                           0.84        92
   macro avg       0.84      0.83      0.84        92
weighted avg       0.84      0.84      0.84        92

0.8369565217391305 accuracy
0.8612689393939394 roc
(0.8353811284742931, 0.8612689393939394) res
0.7507462138376124 50 num_split?
{'C': 100, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1 1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2
 1 1 2 2 2 2 2 2 1 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(92,) EXAMPLE Y
(92, 4131) new
(92,) clone
(92, 4131)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1 1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2
 1 1 2 2 2 2 2 2 1 2 2 

In [31]:
fit_method='use_second'
predict_method='same'
run_combos_list(model_type=SVC,names=name_combinations,
                feature_list=feature_combinations,times=10,
                use_agg=True,agg_fit=fit_method,
                agg_pred=predict_method,y_comb=y_comb,use_nested=False)  
# run_combos_list(model_type=SVC,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,
#                 agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

0.6610407925407925 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4095, 2) new
(46,) clone
(46, 4095, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.68      0.59      0.63        22
           2       0.67      0.75      0.71        24

    accuracy                           0.67        46
   macro avg       0.68      0.67      0.67        46
weighted avg       0.68      0.67      0.67        46

0.6739130434782609 accuracy
0.6988636363636364 roc
(0.6700143472022957, 0.6988636363636364) res
0.6612430347430346 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2

0.6490517815517817 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 36, 2) new
(46,) clone
(46, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.65      0.68      0.67        22
           2       0.70      0.67      0.68        24

    accuracy                           0.67        46
   macro avg       0.67      0.67      0.67        46
weighted avg       0.67      0.67      0.67        46

0.6739130434782609 accuracy
0.6685606060606062 roc
(0.6737588652482269, 0.6685606060606062) res
0.7148080808080808 50 num_split?
{'C': 200, 'class_weight': 'balanced', 'degree': 3, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1

0.6021331446331447 50 num_split?
{'C': 0.3, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 8, 2) new
(46,) clone
(46, 8, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.62      0.45      0.53        22
           2       0.60      0.75      0.67        24

    accuracy                           0.61        46
   macro avg       0.61      0.60      0.60        46
weighted avg       0.61      0.61      0.60        46

0.6086956521739131 accuracy
0.7329545454545454 roc
(0.5964912280701753, 0.7329545454545454) res
0.7379080919080917 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 

0.6438201798201798 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 44, 2) new
(46,) clone
(46, 44, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.70      0.73      0.71        22
           2       0.74      0.71      0.72        24

    accuracy                           0.72        46
   macro avg       0.72      0.72      0.72        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.678030303030303 roc
(0.7172576832151301, 0.678030303030303) res
0.6999666444666444 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 3, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2

0.6541531246531247 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 36, 2) new
(46,) clone
(46, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.67      0.73      0.70        22
           2       0.73      0.67      0.70        24

    accuracy                           0.70        46
   macro avg       0.70      0.70      0.70        46
weighted avg       0.70      0.70      0.70        46

0.6956521739130435 accuracy
0.6969696969696969 roc
(0.6956521739130435, 0.6969696969696969) res
0.6389363414363415 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 

...

0.6733218448218448 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 44, 2) new
(46,) clone
(46, 44, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.70      0.73      0.71        22
           2       0.74      0.71      0.72        24

    accuracy                           0.72        46
   macro avg       0.72      0.72      0.72        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.678030303030303 roc
(0.7172576832151301, 0.678030303030303) res
0.7051486291486291 50 num_split?
{'C': 200, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 

0.6525945720945721 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 36, 2) new
(46,) clone
(46, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.67      0.73      0.70        22
           2       0.73      0.67      0.70        24

    accuracy                           0.70        46
   macro avg       0.70      0.70      0.70        46
weighted avg       0.70      0.70      0.70        46

0.6956521739130435 accuracy
0.6969696969696969 roc
(0.6956521739130435, 0.6969696969696969) res
0.6447641247641246 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 

0.7270188145188144 50 num_split?
{'C': 0.3, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 8, 2) new
(46,) clone
(46, 8, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.77      0.77      0.77        22
           2       0.79      0.79      0.79        24

    accuracy                           0.78        46
   macro avg       0.78      0.78      0.78        46
weighted avg       0.78      0.78      0.78        46

0.782608695652174 accuracy
0.7878787878787878 roc
(0.7821969696969697, 0.7878787878787878) res
0.6492373737373737 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 

0.7355117105117105 50 num_split?
{'C': 200, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4111, 2) new
(46,) clone
(46, 4111, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.79      0.68      0.73        22
           2       0.74      0.83      0.78        24

    accuracy                           0.76        46
   macro avg       0.77      0.76      0.76        46
weighted avg       0.76      0.76      0.76        46

0.7608695652173914 accuracy
0.7916666666666667 roc
(0.7580105212816834, 0.7916666666666667) res
0.7071098346098345 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 3, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 

0.6948495948495947 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4123, 2) new
(46,) clone
(46, 4123, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.78      0.64      0.70        22
           2       0.71      0.83      0.77        24

    accuracy                           0.74        46
   macro avg       0.75      0.73      0.73        46
weighted avg       0.74      0.74      0.74        46

0.7391304347826086 accuracy
0.7518939393939394 roc
(0.7346153846153847, 0.7518939393939394) res
0.707018204018204 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 

In [45]:
conditionals = ((full_stat_df['CogTr']==1))
feature_combinations,name_combinations,y_comb=run_embedding_classiciation(full_stat_df,X_weight,skip_w=True,
                                                                   use_prob=False,min_combos=1,max_combos=4,conditionals=conditionals)

conditational
0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    False
179    False
Name: CogTr, Length: 180, dtype: bool
NO SHUFFLE
conditational
0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    False
179    False
Name: CogTr, Length: 180, dtype: bool
NO SHUFFLE
[[1.14856664 2.0953061  2.18031072 ... 2.05160935 1.42537626 2.26457091]
 [1.12309426 2.06000353 2.21085388 ... 2.09441905 1.45372466 2.33475558]
 [1.21732124 1.98617806 2.04157465 ... 1.89624364 1.33290461 1.99036129]
 ...
 [1.19672428 2.10935528 2.21695379 ... 2.04229307 1.52801455 2.00044454]
 [1.30283475 1.89239307 1.91787272 ... 1.98593059 1.42585129 1.99628657]
 [1.29968796 1.78858962 1.84196672 ... 1.9000061  1.39210995 1.90546139]] NF
conditational
0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    Fal

In [42]:
fit_method='concatenate'
predict_method='same'
run_combos_list(model_type=SV,names=name_combinations,
                feature_list=feature_combinations,times=1,
                use_agg=True,agg_fit=fit_method,
                agg_pred=predict_method,y_comb=y_comb,use_nested=False)  
# run_combos_list(model_type=SVC,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,
#                 agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

ValueError: Invalid parameter probability for estimator LogisticRegression(). Check the list of available parameters with `estimator.get_params().keys()`.

In [38]:
fit_method='split_sample'
predict_method='prediction_mean'
run_combos_list(model_type=SVC,names=name_combinations,
                feature_list=feature_combinations,times=1,
                use_agg=True,agg_fit=fit_method,
                agg_pred=predict_method,y_comb=y_comb,use_nested=False)  
# run_combos_list(model_type=SVC,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,
#                 agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

0.5834666507756763 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 1 1 2 1 1 1 1 2 1 2 2 1 2 2 2 2 2 2 1 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 1 1 1 2 1 1 2 2 2 2 1 2 1 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 1 1 1 1 1 1 2 2
 2 1 1 2 2 2 1 1 2 2 2 2 2 1 1 1] EXAMPLE Y
(90,) EXAMPLE Y
(90, 4095, 2) new
(90,) clone
(90, 4095, 2)  X shape???sss
LeaveOneOut()
[1 2 2 1 1 2 1 1 1 1 2 1 2 2 1 2 2 2 2 2 2 1 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 1 1 1 2 1 1 2 2 2 2 1 2 1 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 1 1 1 1 1 1 2 2
 2 1 1 2 2 2 1 1 2 2 2 2 2 1 1 1] YYY
              precision    recall  f1-score   support

           1       0.60      0.61      0.60        41
           2       0.67      0.65      0.66        49

    accuracy                           0.63        90
   macro avg       0.63      0.63      0.63        90
weighted avg       0.63      0.63      0.63        90

0.6333333333333333 accuracy
0.6326530612244898 roc
(0.6311017264936033, 0.6326530

              precision    recall  f1-score   support

           1       0.57      0.56      0.57        41
           2       0.64      0.65      0.65        49

    accuracy                           0.61        90
   macro avg       0.61      0.61      0.61        90
weighted avg       0.61      0.61      0.61        90

0.6111111111111112 accuracy
0.525634644101543 roc
(0.6071829405162739, 0.525634644101543) res
0.5618023228099281 50 num_split?
{'C': 200, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 1 1 2 1 1 1 1 2 1 2 2 1 2 2 2 2 2 2 1 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 1 1 1 2 1 1 2 2 2 2 1 2 1 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 1 1 1 1 1 1 2 2
 2 1 1 2 2 2 1 1 2 2 2 2 2 1 1 1] EXAMPLE Y
(90,) EXAMPLE Y
(90, 36, 2) new
(90,) clone
(90, 36, 2)  X shape???sss
LeaveOneOut()
[1 2 2 1 1 2 1 1 1 1 2 1 2 2 1 2 2 2 2 2 2 1 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 1 1 1 2 1 1 2 2 2 2 1 2 1 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 1 1 1 1 1 1 2 2
 2 1 1 2 2 2 1 1 2 2 2 2 2 

In [44]:
fit_method='split_sample'
predict_method='use_second'
run_combos_list(model_type=SVC,names=name_combinations,
                feature_list=feature_combinations,times=1,
                use_agg=True,agg_fit=fit_method,
                agg_pred=predict_method,y_comb=y_comb,use_nested=False)  
# run_combos_list(model_type=SVC,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,
#                 agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

0.6862872127872127 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4095, 2) new
(46,) clone
(46, 4095, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.69      0.82      0.75        22
           2       0.80      0.67      0.73        24

    accuracy                           0.74        46
   macro avg       0.75      0.74      0.74        46
weighted avg       0.75      0.74      0.74        46

0.7391304347826086 accuracy
0.7178030303030303 roc
(0.7386363636363636, 0.7178030303030303) res
0.5901421911421911 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1

0.629474913974914 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4131, 2) new
(46,) clone
(46, 4131, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.65      0.59      0.62        22
           2       0.65      0.71      0.68        24

    accuracy                           0.65        46
   macro avg       0.65      0.65      0.65        46
weighted avg       0.65      0.65      0.65        46

0.6521739130434783 accuracy
0.6401515151515151 roc
(0.6495238095238096, 0.6401515151515151) res
0.6585864135864135 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1

In [47]:
fit_method='use_second'
predict_method='same'
run_combos_list(model_type=SVC,names=name_combinations,
                feature_list=feature_combinations,times=10,
                use_agg=True,agg_fit=fit_method,
                agg_pred=predict_method,y_comb=y_comb,use_nested=False)  
# run_combos_list(model_type=SVC,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,
#                 agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

0.631494227994228 50 num_split?
{'C': 100, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4095, 2) new
(46,) clone
(46, 4095, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.72      0.59      0.65        22
           2       0.68      0.79      0.73        24

    accuracy                           0.70        46
   macro avg       0.70      0.69      0.69        46
weighted avg       0.70      0.70      0.69        46

0.6956521739130435 accuracy
0.678030303030303 roc
(0.6903846153846154, 0.678030303030303) res
0.7159473304473303 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1

0.6602296592296593 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4131, 2) new
(46,) clone
(46, 4131, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.72      0.59      0.65        22
           2       0.68      0.79      0.73        24

    accuracy                           0.70        46
   macro avg       0.70      0.69      0.69        46
weighted avg       0.70      0.70      0.69        46

0.6956521739130435 accuracy
0.7159090909090909 roc
(0.6903846153846154, 0.7159090909090909) res
0.6983175158175159 50 num_split?
{'C': 100, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 

0.6879376734376733 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4123, 2) new
(46,) clone
(46, 4123, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.74      0.64      0.68        22
           2       0.70      0.79      0.75        24

    accuracy                           0.72        46
   macro avg       0.72      0.71      0.71        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.7386363636363636 roc
(0.7140124342419896, 0.7386363636363636) res
0.7109953934953934 50 num_split?
{'C': 100, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2

0.679505994005994 50 num_split?
{'C': 100, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4095, 2) new
(46,) clone
(46, 4095, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.72      0.59      0.65        22
           2       0.68      0.79      0.73        24

    accuracy                           0.70        46
   macro avg       0.70      0.69      0.69        46
weighted avg       0.70      0.70      0.69        46

0.6956521739130435 accuracy
0.678030303030303 roc
(0.6903846153846154, 0.678030303030303) res
0.7082536352536353 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1

0.6696460206460206 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4131, 2) new
(46,) clone
(46, 4131, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.72      0.59      0.65        22
           2       0.68      0.79      0.73        24

    accuracy                           0.70        46
   macro avg       0.70      0.69      0.69        46
weighted avg       0.70      0.70      0.69        46

0.6956521739130435 accuracy
0.7159090909090909 roc
(0.6903846153846154, 0.7159090909090909) res
0.6962251082251083 50 num_split?
{'C': 100, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 

...

0.6868620268620268 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4131, 2) new
(46,) clone
(46, 4131, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.72      0.59      0.65        22
           2       0.68      0.79      0.73        24

    accuracy                           0.70        46
   macro avg       0.70      0.69      0.69        46
weighted avg       0.70      0.70      0.69        46

0.6956521739130435 accuracy
0.7159090909090909 roc
(0.6903846153846154, 0.7159090909090909) res
0.6865045510045509 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2

0.6982089022089022 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4123, 2) new
(46,) clone
(46, 4123, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.74      0.64      0.68        22
           2       0.70      0.79      0.75        24

    accuracy                           0.72        46
   macro avg       0.72      0.71      0.71        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.7386363636363636 roc
(0.7140124342419896, 0.7386363636363636) res
0.7046951936951936 50 num_split?
{'C': 100, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2

0.6197590742590742 50 num_split?
{'C': 100, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4095, 2) new
(46,) clone
(46, 4095, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.72      0.59      0.65        22
           2       0.68      0.79      0.73        24

    accuracy                           0.70        46
   macro avg       0.70      0.69      0.69        46
weighted avg       0.70      0.70      0.69        46

0.6956521739130435 accuracy
0.678030303030303 roc
(0.6903846153846154, 0.678030303030303) res
0.6938248418248417 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 

0.6705396825396825 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4131, 2) new
(46,) clone
(46, 4131, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.72      0.59      0.65        22
           2       0.68      0.79      0.73        24

    accuracy                           0.70        46
   macro avg       0.70      0.69      0.69        46
weighted avg       0.70      0.70      0.69        46

0.6956521739130435 accuracy
0.7159090909090909 roc
(0.6903846153846154, 0.7159090909090909) res
0.7064751359751359 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2

0.7042616272616273 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4123, 2) new
(46,) clone
(46, 4123, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.74      0.64      0.68        22
           2       0.70      0.79      0.75        24

    accuracy                           0.72        46
   macro avg       0.72      0.71      0.71        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.7386363636363636 roc
(0.7140124342419896, 0.7386363636363636) res
0.6864265179265181 50 num_split?
{'C': 10, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 

In [43]:

run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=2,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb,use_nested=True)  

ValueError: Invalid parameter probability for estimator LogisticRegression(). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
feature_combinations,name_combinations,y_comb=run_embedding_classiciation(all_stat_dicts[2],X_weight,
                                                                   use_prob=True,min_combos=3,max_combos=4)

In [ ]:
run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=2,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb,use_nested=True)  

In [ ]:
skks
feature_combinations,name_combinations,y_comb=run_embedding_classiciation(all_stat_dicts[2],X_weight,
                                                                   use_prob=True,min_combos=3,max_combos=4)

In [ ]:
run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=1,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

In [ ]:
feature_combinations,name_combinations,y_comb=run_embedding_classiciation(full_dict,X_weight,
                                                                   use_prob=True,min_combos=3,max_combos=4)

In [ ]:
run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=3,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

In [ ]:
num=1
names_short=name_combinations[:num]
feats_short=feature_combinations[:num]

In [ ]:
run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=1,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb)  